In [1]:
import sys,os

import pandas as pd
import numpy as np
import OutcomesGen as og
import seaborn as sns
from glob import glob
from multiprocessing import Pool
from matplotlib import pyplot as plt
from time import sleep

from IPython.display import clear_output

sns.set()

# CDCFormatter

This notebook processes experimental exposed by subpopulation and day data to generate full outcome data across ages, geographies, and disease states:

* Disease state transitions are loaded from *FluTransitionsP3.xlsx*
* Breakdown of ages by region is loaded from *../PatchSim-Experiments-Gen/FixedUSAPop.patchsim*
* Membership of geographies by counties' first 2 FIPS digits is loaded from *SubsetRegionFIPS.csv*
* CDC output templates are loaded from *templates/{experiment-outcome}.csv*
* Experimental parameters are loaded from *../PatchSim-Experiments-Gen/experiments/{experiment}/MetaData.csv*
* The method **runAllOutcomes()** uses said metadata to load experiment scenario outputs, stochastically transition exposures between disease states, break down disease states by age and geography, and format the resulting curves in accordance to the CDC templates
* Generated data and figures are saved to the *output* and *figures* subdirectories for each scenario in *../PatchSim-Experiments-Gen/experiments/{experiment}/* 

Note: If **runAllOutcomes()** is run with the parameter **skipFinished=True**, then outcome processing will skip scenario subdirectories within experiments which contain an output directory. This allows for multiple instances of the notebook may be ran simultaneously without interferenceto distribute outcome processing workloads across machines.


### Globals

In [2]:
ageKey = {'all':'Overall',
           'p':'0-4 years',
           's':'5-17 years',
           'a':'18-49 years',
           'o':'50-64 years',
           'g':'65+ years'}



def getRegionPopSizes(ref='../PatchSim-Experiments-Gen/FixedUSAPop.patchsim'):
    """Loads region pop size data"""
    sizeDf = pd.read_csv(ref,
                         sep=' ',
                         names=['population','popsize'])
    sizeDf.loc[:,'fips'] = sizeDf.population.str[:-1]
    sizeDf.loc[:,'age_group'] = sizeDf.population.str[-1]
    sizeDf.set_index(['population'],inplace=True)

    
    regionPopSizes = []
    
    for region,fips in regions.items():
        selectedPops = sizeDf.index.tolist()
        if region != 'US National':
            selectedPops = [population for population in selectedPops if population[:2] in fips]
        regionDf = sizeDf.loc[selectedPops,:].copy(deep=True)
        cohortSizes = dict(regionDf.groupby(['age_group']).popsize.sum())
        cohortSizes['all'] = regionDf.popsize.sum()
        regionPopSizes.append({**{'region':region},
                               **{'all':regionDf.popsize.sum()},
                               **cohortSizes})
        
    regionPopSizes = pd.DataFrame(regionPopSizes)    
    regionPopSizes.set_index(['region'],inplace=True)
        
        
    return regionPopSizes
        


def getRegions():
    """Get population regions"""
    df = pd.read_csv('SubsetRegionFIPS.csv')
    regions = {region:set(members.split()) for region, members in zip(df.region,df.members)}
    return regions
    
regions = getRegions()
regionPopSizes = getRegionPopSizes()

In [3]:
regionPopSizes

,all,a,g,o,p,s
region,,,,,,
US National,296710391,119404302.0,45765003.0,61929159.0,17766046.0,51845881.0
HHS Region 1,13718580,5356659.0,2289674.0,3160893.0,708510.0,2202844.0
HHS Region 2,26527179,10808654.0,4151408.0,5694207.0,1486332.0,4386578.0
HHS Region 3,28361274,11241648.0,4615006.0,6227278.0,1610763.0,4666579.0
HHS Region 4,58234780,22730924.0,9935548.0,12352160.0,3341164.0,9874984.0
...,...,...,...,...,...,...
Wisconsin,5527167,2142535.0,871148.0,1232873.0,320531.0,960080.0
Wyoming,551298,214056.0,83061.0,119598.0,36779.0,97804.0
District of Columbia,570286,295960.0,75013.0,100099.0,31943.0,67271.0


### Data & template loaders

In [4]:
def getTemplate(ref='templates/HP09_SymIllness_NEU.csv'):
    """Loads a CDC output format template"""
    template = pd.read_csv(ref)
    bins = template.Bin.unique()
    cmlBins = template.Bin_cml.unique()
    
    getRange = lambda x: [float(i) for i in x.replace('[','').replace(']','').replace(')','').replace('Inf','9999').split('-')]
    ranges = {i:getRange(i) for i in bins if i.startswith('[')}
    cmlRanges = {i:getRange(i) for i in cmlBins if i.startswith('[')}
    
    percs = {'sm.mean':np.mean,
            'sm.median':np.median,
            'sm.perc2p5':lambda x: np.percentile(x,2.5),
            'sm.perc5':lambda x: np.percentile(x,5),
            'sm.perc25':lambda x: np.percentile(x,25),
            'sm.perc75':lambda x: np.percentile(x,75),
            'sm.perc95':lambda x: np.percentile(x,95),
            'sm.perc97p5':lambda x: np.percentile(x,97.5),
            'sm.peak':np.max}
    
    return ranges, cmlRanges, percs

### Initial run data prep

In [5]:
def prepOne(args):
    """Worker subprocess for prepRuns()"""
    (splitRun,sample,scenario,approximateAges,transitionsRef,scalers) = args
    print("Starting sample",sample)
    splitRun = splitRun.drop(['sample'],axis=1).set_index('id').T
    splitRun.index = splitRun.index.astype('int32')
    splitRun.sort_index(inplace=True)
    
    
    outcomes = og.getCOVIDOutcomes(splitRun,
                                    scenario,
                                    ignore=set(),
                                    dwellFields=set(),
                                    approximateAges=False,
                                    mergeAges=False,
                                    mergeStates=True,
                                    saveResults=False,
                                    noThreading=True,
                                    verbose=False,
                                    flowsRef=transitionsRef,
                                    scalers=scalers)
    
    outcomes = og.autoMerge(outcomes,toForce={'Hosp',
                                              'Isymp',
                                              'AVDoses',
                                              'Vent',
                                              'Death'})

    toKeep = {'E',
              'Hosp',
              'AVDoses',
              'Isymp',
              'Vent',
              'Death'}
    
    outcomes = {key:value for key,value in outcomes.items() if key in toKeep}
    
    selectedRows = splitRun.index
    for key,compartmentDf in outcomes.items():
        compartmentDf = compartmentDf.loc[selectedRows,:]
        compartmentDf.index = 'Week'+(compartmentDf.index.astype(int)/7+1).astype(int).astype(str).str.zfill(2)
        compartmentDf = compartmentDf.groupby(compartmentDf.index).sum().sort_index()
        compartmentDf.index = [i.replace('Week0','Week') for i in compartmentDf.index]
        outcomes[key] = compartmentDf
 
    return outcomes


def prepRuns(ref,
            threads=5,
            scenario='default',
            approximateAges=False,
            scalers='null',
            transitionsRef='FluTransitionsP3.xlsx'):
    """Generates disease state data from processing of simulation output through transition model"""
    mergedData = pd.read_csv(ref)
    samples = sorted(list(mergedData['sample'].unique()))
    
    splitRun = mergedData[mergedData['sample'] == 1]
    splitRun = splitRun.drop(['sample'],axis=1).set_index('id')
    print("Loading transitions from",transitionsRef)
    og.getTransitions(transitionsRef,scenario=scenario)
    
    print("Loading %s, found %s samples" % (ref,len(samples)))
    toRun = [(mergedData[mergedData['sample'] == sample].copy(deep=True),
              sample,
              scenario,
              approximateAges,
              transitionsRef,
              scalers) for sample in samples]
    
    
    if threads == 1:
        splitRuns = [prepOne(i) for i in toRun]
    else:
        p = Pool(threads)
        splitRuns = p.map(prepOne,toRun)
        p.close()
        p.join()
    
    print("Single experiment prep done.")
    return splitRuns

  

### Formatter execution

In [6]:
def plotRawSpread(dfIn,
                  outcome,
                  title='',
                  yLabel='',
                  figFile=''):
    """Debugging method to plot curves before formatting"""
    
    color = {'E':'seagreen',
                'Isymp':'orange',
                'Hosp':'orangered',
                'Vent':'darkred',
                'Death':'black',
                'AVDoses':'blueviolet'}[outcome]
    df = dfIn.copy(deep=True)
    df.loc[:,'Week of simulation'] = df.index
    df = pd.melt(df,id_vars='Week of simulation',var_name='run')
    df['Week of simulation'] = df['Week of simulation'].str.replace("Week",'').astype(int)
    fig,ax = plt.subplots(figsize=(8,6))

    sns.lineplot(x="Week of simulation",
                 y="value",
                 data=df,
                 ax=ax,
                 color=color)
    ax.set_ylabel(yLabel)
    ax.set_title(title)
    
    if figFile != '':
        plt.savefig(figFile,bbox_inches='tight')
    plt.close()
        

def binCmlCurves(cmlRanges,percs,percsByGeo):
    """Bins cumulative curves"""
    nRuns = len(percsByGeo)
    cmlRangeBins = np.array([i[0] for i in cmlRanges.values()])
    cmlRangeNames = list(cmlRanges.keys())
    
    cmlRangesDf = pd.DataFrame(0.,
                               index=cmlRanges.keys(),
                               columns=['cml'])
    
    cmlInfs = [run.sum() for run in percsByGeo]

    inds = [cmlRangeNames[i[0]] for i in list(np.digitize(cmlInfs,cmlRangeBins)-1)]
    for ind in inds:
        cmlRangesDf.at[ind,'cml'] += 1
    cmlRangesDf['cml'] = cmlRangesDf['cml'] / nRuns
        
    percRangesDf = pd.DataFrame(0.,
                               index=percs.keys(),
                               columns=['cml'])
    
    mergedCml = pd.concat([cmlRangesDf,percRangesDf])
    
    return mergedCml

        

        
def formatEntry(runDataIn,
                outcome,
                ageCohort='all',
                showPlot=True,
                template='templates/HP09_SymIllness_NEU.csv',
                dirOut='test',
                fileName='test',
                selectedGeo='US National',
                plotRaw=False,
                plotCmlSpread=False,
                units=100):
    """Generates outcome data and figures for one age cohort, disease outcome, and geography"""
    
    runData = [outcomesDict[outcome].copy(deep=True) for outcomesDict in runDataIn]
    
    if ageCohort == 'all':
        selectedPops = list(runData[0].columns)
    else:
        selectedPops = [i for i in runData[0].columns if i.endswith(ageCohort)]
        
    if selectedGeo != 'US National':
        selectedPops = [i for i in selectedPops if i[:2] in regions[selectedGeo]]

    regionAbbrev = selectedGeo.replace(' ','')

    geoMapper = {i:selectedGeo for i in runData[0].columns}    
        
    runData = [run[selectedPops] for run in runData]
    
    popSize = regionPopSizes.at[selectedGeo,ageCohort]
    print("Running with popsize of",popSize)
        
    ranges, cmlRanges, percs = getTemplate(template)
    geoGraphies = geoMapper.values()
    runsByGeo = [run.rename(geoMapper,axis=1).groupby(axis=1, level=0).sum() for run in runData]
    percsByGeo = [(units*run)/popSize for run in runsByGeo]
    weeks = list(runData[0].index)
    nRuns = len(runData)
        
    percsByWeek = pd.DataFrame(index=percs.keys(),
                               columns=weeks)
    rangesByWeek = pd.DataFrame(0,
                               index=ranges.keys(),
                               columns=weeks)

    rangeBins = np.array([i[0] for i in ranges.values()])
    rangeNames = list(ranges.keys())
    
    rawCurves = pd.DataFrame({i:curve.iloc[:,0] for i,curve in enumerate(percsByGeo)})
    
    for run in percsByGeo:
        inds = [rangeNames[i[0]] for i in list(np.digitize(run,rangeBins)-1)]
        for week,ind in zip(weeks,inds):
            rangesByWeek.at[ind,week] += 1
                        
    rangesByWeek = rangesByWeek/nRuns
    yUnitLabel = {100:'Percent in compartment bin',
                 100000:'Number in compartment bin per 100k persons'}[units]
    
    fig,ax = plt.subplots(figsize=(8,8))
    rangesPlotDf = rangesByWeek.copy(deep=True)
    rangesPlotDf = rangesPlotDf[rangesPlotDf.index <= rangesPlotDf[rangesPlotDf.sum(axis=1) > 0].index[-1]]
    
    sns.heatmap(rangesPlotDf[::-1],
                cmap='viridis_r',
                ax=ax)
    scenario = dirOut.split('/')[-2]
    figFile = '%sfigures/%s/%s_%s_%s.png' % (dirOut,regionAbbrev,fileName,regionAbbrev,ageCohort)
    plotTitle = 'UVA %s %s age: %s' % (scenario,selectedGeo,ageKey[ageCohort])

    plt.title(plotTitle)
    plt.xlabel('Week')
    plt.ylabel(yUnitLabel)
    plt.savefig(figFile,bbox_inches='tight')
    
    if ageCohort == 'all':
        plt.show()
    plt.close()
    
    if plotRaw:
        plotRawSpread(rawCurves,
                      outcome,
                      title=plotTitle,
                      figFile=figFile.replace('.png','_curves.png'),
                      yLabel=yUnitLabel)
    
    for week in weeks:
        weekValues = [float(run.at[week,selectedGeo]) for run in percsByGeo]
        for metric,fx in percs.items():
            percsByWeek.at[metric,week] = fx(weekValues)
    

    rangesByWeek.at[:,'Peak Magnitude'] = 0.
    peaks = rawCurves.max(axis=0)
    inds = [rangeNames[i] for i in list(np.digitize(peaks,rangeBins)-1)]
    increment = 1/nRuns
    for ind in inds:
        rangesByWeek.at[ind,'Peak Magnitude'] += increment
    for metric,fx in percs.items():
        percsByWeek.at[metric,'Peak Magnitude'] = fx(peaks)
    
    
    mergedTables = pd.concat([rangesByWeek,percsByWeek])
    mergedTables.loc[:,'Bin'] = mergedTables.index
    cmlDf = binCmlCurves(cmlRanges,percs,percsByGeo)
    mergedTables.loc[:,'Bin_cml'] = list(cmlDf.index)
    mergedTables.loc[:,'Cml'] = list(cmlDf['cml'])
    mergedTables.index = range(len(mergedTables))
    mergedTables.loc[:,'Location'] = selectedGeo
    mergedTables.loc[:,'Agegroup'] = ageKey[ageCohort]
    
    
    frontCols = ['Location','Agegroup','Bin_cml','Cml','Bin','Peak Magnitude']
    mergedTables = mergedTables[frontCols+weeks]
    
    cmlPlotDf = mergedTables.iloc[:len(rangesByWeek)][['Bin_cml','Cml']].copy(deep=True)
    cmlPlotDf.set_index("Bin_cml",inplace=True)
    cmlPlotDf = cmlPlotDf[cmlPlotDf.index <= cmlPlotDf[cmlPlotDf.Cml != 0].index[-1]]
    cmlPlotDf = cmlPlotDf[cmlPlotDf.index >= cmlPlotDf[cmlPlotDf.Cml != 0].index[0]]

    if plotCmlSpread:
        fig,ax = plt.subplots(figsize=(10,6))
        cmlPlotDf.Cml.plot(kind='bar',ax=ax)
        cmlTitle = plotTitle + 'Cml Distribution'
        cmlFile = figFile.replace('.png','_CmlDist.png')
        plt.xlabel('Bin_cml')
        plt.ylabel('Proportion')
        plt.title(plotTitle)
        plt.ylim(0,cmlPlotDf.Cml.max())
        plt.savefig(cmlFile,bbox_inches='tight')

        plt.close()
    
    return mergedTables, rawCurves



def runAllOutcomes(metaRef,
                   transitionsRef='FluTransitionsP3.xlsx',
                   templatesRef='TemplateAlignmentRLV6.csv',
                   skipFinished=True,
                   scalers='null',
                   dumpRaw=False,
                   makeOutcomes=True):
    """Primary analysis function, parses experiment metadata to execute full outcome processing"""
    metaDf = pd.read_csv(metaRef)
    
    outcomes = ['Isymp',
                'Hosp',
                'Vent',
                'Death',
                'AVDoses',]
    
    templates = pd.read_csv(templatesRef,index_col=0)
        
    for index,row in metaDf.iterrows():
        dataFile = row['MergedOutput']
        scenario = row['Scenario']
        dirOut = dataFile.replace("MergedSamples.csv",'outcomes/')
        figsOut = dataFile.replace("MergedSamples.csv",'figures/')
        isFinished = len(glob(dirOut+'*_AntiviralTX_UVA.csv')) == 1
        isStarted = os.path.exists(dirOut)
        
        if not isStarted or not skipFinished:
            if not os.path.exists(dirOut):
                os.makedirs(dirOut)
            if not os.path.exists(figsOut):
                os.makedirs(figsOut)
            interventions = row['Interventions']
            avUsed = '!AV' not in interventions
            opScenario = {True:'av',
                          False:'default'}[avUsed]
            while not os.path.exists(dataFile):
                print("Datafile %s not found, sleeping" % dataFile)
                sleep(60)
            
            global runsGlobal
            runsGlobal = prepRuns(dataFile,
                            scenario=opScenario,
                            threads=10,
                            transitionsRef=transitionsRef,
                            scalers=scalers)
            
            if dumpRaw:
                rawOutRef = dataFile.replace("MergedSamples.csv",'RawOutcomes.pickle')
                with open(rawOutRef,'wb') as fileOut:
                    pickle.dump(runsGlobal,
                                fileOut,
                                protocol=-1)
            if makeOutcomes:
                for outcome in outcomes:
                    template = templates.at[outcome,'template']
                    fileName = templates.at[outcome,'fileOut']
                    units = templates.at[outcome,'units']
                    ageTables = []
                    fileOut = '%s%s_%s_UVA.csv' % (dirOut,scenario,fileName)
                    fileName = '%s_%s_UVA' % (scenario,fileName)

                    for region in regions.keys():
                        regionAbbrev = region.replace(' ','')
                        regionDataDir = '%s%s/' % (dirOut,regionAbbrev)
                        regionFigDir = '%s%s/' % (figsOut,regionAbbrev)
                        if not os.path.exists(regionDataDir):
                            os.makedirs(regionDataDir)
                        if not os.path.exists(regionFigDir):
                            os.makedirs(regionFigDir)

                        ageRunArgs = []

                        for age in ['all','p','s','a','o','g']:
                            ageRunArgs.append((age,
                                               outcome,
                                               template,
                                               dataFile.replace("MergedSamples.csv",'/'),
                                               fileName,
                                               units,
                                               region))


                        p = Pool(6)
                        splitRuns = p.map(formatEntrySub,ageRunArgs)
                        p.close()
                        p.join()

                        ageTables += [i for i in splitRuns if type(i) is not str]
                        clear_output()

                    pd.concat(ageTables).to_csv(fileOut,index=False)

        

def formatEntrySub(args):
    """Subprocess to run age cohort outcome processing in parallel for a single disease state and geo"""
    (age,outcome,template,dirOut,fileName,units,region) = args
    regionAbbrev = region.replace(' ','')
    
    try:
        mergedTables, rawCurves = formatEntry(runsGlobal,
                                              ageCohort=age,
                                              outcome=outcome,
                                              showPlot=True,
                                              template=template,
                                              dirOut=dirOut,
                                              fileName=fileName,
                                              units=units,
                                              selectedGeo=region)
        
        rawOut = '%soutcomes/%s/%s_%s_RawCurves.csv' % (dirOut,regionAbbrev,fileName,age)
        rawCurves.to_csv(rawOut)
        return mergedTables
    except:
        print("Failed for",region,age)
        return 'failed'
    

#### Run hypothetical scenario outcome processing

In [7]:
runAllOutcomes('../PatchSim-Experiments-Gen/experiments/WorkingTemplateP9_USA/MetaData.csv',
              transitionsRef='FluTransitionsP3.xlsx',
              templatesRef='TemplateAlignmentHPV6.csv',
              skipFinished=True,
              scalers='null')

#### Run 2009 scenario outcome processing

In [ ]:
runAllOutcomes('../PatchSim-Experiments-Gen/experiments/WorkingTemplateP9_USA/MetaData.csv',
              transitionsRef='FluTransitionsP3.xlsx',
              skipFinished=False,
              scalers='null')